# TLDR

In this notebook, we compare different methods and training data types to use for fitting the LMC model. 

In [1]:
%load_ext autoreload
%autoreload 2

from typing import Optional
import copy
import torch
from torch import Tensor
import gpytorch
from botorch.fit import fit_gpytorch_model, fit_gpytorch_mll, fit_gpytorch_mll_scipy
from botorch.models.gpytorch import GPyTorchModel
from botorch.models.model import Model
from gpytorch import ExactMarginalLogLikelihood
from gpytorch.kernels import Kernel, LCMKernel, MaternKernel, RBFKernel, ScaleKernel
from gpytorch.likelihoods import Likelihood, MultitaskGaussianLikelihood
from gpytorch.models import ExactGP
from gpytorch.priors.torch_priors import Prior, GammaPrior
from gpytorch.priors import SmoothedBoxPrior
from gpytorch.constraints import GreaterThan, Interval
from gpytorch.priors.lkj_prior import LKJCovariancePrior


from botorch.models.transforms.outcome import OutcomeTransform, Standardize
from botorch.models.transforms.input import InputTransform



/home/yz685/anaconda3/envs/bope_pca/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# LCM model class
class MultitaskGPModel(GPyTorchModel, ExactGP):
    def __init__(
        self,
        train_X: Tensor,
        train_Y: Tensor,
        latent_dim: int,
        rank: Optional[int] = None,
        task_covar_prior: Optional[Prior] = None,
        likelihood: Optional[MultitaskGaussianLikelihood] = None,
        outcome_transform: Optional[OutcomeTransform] = None,
        input_transform: Optional[InputTransform] = None,
    ):

        r"""
        Initialize model class for multi-output GP models.

        Args:
            train_X: `num_samples x input_dim` tensor
            train_Y: `num_samples x outcome_dim` tensor
            latent_dim: number of basis kernels to use
            outcome_transform: OutcomeTransform
            input_transform: InputTransform

        """

        with torch.no_grad():
            transformed_X = self.transform_inputs(
                X=train_X, input_transform=input_transform
            )
        if outcome_transform is not None:
            train_Y, _ = outcome_transform(train_Y)
        if rank is None:
            rank = 1
        self._validate_tensor_args(X=transformed_X, Y=train_Y)
        self._num_outputs = train_Y.shape[-1]
        
        ard_num_dims, num_tasks = train_X.shape[-1], train_Y.shape[-1]

        if task_covar_prior is None:
            sd_prior = GammaPrior(1.0, 0.15)
            eta = 0.5
            task_covar_prior = LKJCovariancePrior(num_tasks, eta, sd_prior)
    
        if likelihood is None:            
            likelihood = MultitaskGaussianLikelihood(
                num_tasks=num_tasks)

        super().__init__(
            train_inputs=train_X, train_targets=train_Y, likelihood=likelihood
        )

        self.mean_module = gpytorch.means.MultitaskMean(
            gpytorch.means.ConstantMean(), num_tasks=num_tasks
        )

        self.covar_module = LCMKernel(
            base_kernels=[MaternKernel(
                nu = 2.5,
                ard_num_dims = ard_num_dims,
                lengthscale_prior = GammaPrior(3.0, 6.0)
            )] * latent_dim,
            num_tasks=num_tasks,
            rank=rank,
            # task_covar_prior=task_covar_prior,
        )

        self.to(train_X)


    def forward(self, x: Tensor):
        r"""
        Return posterior distribution at new point x
        """
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)


## data in `torch.double`, use `fit_gpytorch_model()`

In [46]:
train_X = torch.randn((10,1))
train_Y = torch.randn((10,3))

outcome_model = MultitaskGPModel(
    train_X.to(torch.double),
    train_Y.to(torch.double),
    latent_dim = 1,
    outcome_transform=Standardize(3),
).to(torch.double)
lcm_mll = ExactMarginalLogLikelihood(
    outcome_model.likelihood, outcome_model
)

fit_gpytorch_model(lcm_mll)

/home/yz685/anaconda3/envs/bope_pca/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(


ExactMarginalLogLikelihood(
  (likelihood): MultitaskGaussianLikelihood(
    (raw_task_noises_constraint): GreaterThan(1.000E-04)
    (raw_noise_constraint): GreaterThan(1.000E-04)
  )
  (model): MultitaskGPModel(
    (likelihood): MultitaskGaussianLikelihood(
      (raw_task_noises_constraint): GreaterThan(1.000E-04)
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
    (mean_module): MultitaskMean(
      (base_means): ModuleList(
        (0): ConstantMean()
        (1): ConstantMean()
        (2): ConstantMean()
      )
    )
    (covar_module): LCMKernel(
      (covar_module_list): ModuleList(
        (0): MultitaskKernel(
          (task_covar_module): IndexKernel(
            (IndexKernelPrior): LKJCovariancePrior()
            (raw_var_constraint): Positive()
          )
          (data_covar_module): MaternKernel(
            (lengthscale_prior): GammaPrior()
            (raw_lengthscale_constraint): Positive()
          )
        )
      )
    )
  )
)

## data in `torch.float`, use `fit_gpytorch_model()`

In [47]:
train_X = torch.randn((10,1))
train_Y = torch.randn((10,3))

outcome_model = MultitaskGPModel(
    train_X,
    train_Y,
    latent_dim = 1,
    outcome_transform=Standardize(3),
).to(torch.double)
lcm_mll = ExactMarginalLogLikelihood(
    outcome_model.likelihood, outcome_model
)

fit_gpytorch_model(lcm_mll)

/home/yz685/anaconda3/envs/bope_pca/lib/python3.9/site-packages/botorch/models/gpytorch.py:119: UserWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  warnings.warn(
/home/yz685/anaconda3/envs/bope_pca/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(


ExactMarginalLogLikelihood(
  (likelihood): MultitaskGaussianLikelihood(
    (raw_task_noises_constraint): GreaterThan(1.000E-04)
    (raw_noise_constraint): GreaterThan(1.000E-04)
  )
  (model): MultitaskGPModel(
    (likelihood): MultitaskGaussianLikelihood(
      (raw_task_noises_constraint): GreaterThan(1.000E-04)
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
    (mean_module): MultitaskMean(
      (base_means): ModuleList(
        (0): ConstantMean()
        (1): ConstantMean()
        (2): ConstantMean()
      )
    )
    (covar_module): LCMKernel(
      (covar_module_list): ModuleList(
        (0): MultitaskKernel(
          (task_covar_module): IndexKernel(
            (IndexKernelPrior): LKJCovariancePrior()
            (raw_var_constraint): Positive()
          )
          (data_covar_module): MaternKernel(
            (lengthscale_prior): GammaPrior()
            (raw_lengthscale_constraint): Positive()
          )
        )
      )
    )
  )
)

## data in `torch.double`, use `fit_gpytorch_mll()`

In [5]:
# data type double, 

train_X = torch.randn((10,1), dtype = torch.double)
train_Y = torch.randn((10,3), dtype = torch.double)

outcome_model = MultitaskGPModel(
    train_X,
    train_Y,
    latent_dim = 1,
    outcome_transform=Standardize(3),
)

lcm_mll = ExactMarginalLogLikelihood(
    outcome_model.likelihood, outcome_model
)

fit_gpytorch_mll(lcm_mll)

/home/yz685/anaconda3/envs/bope_pca/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(


RuntimeError: Must provide inverse transform to be able to sample from prior.

## data in `torch.float`, use `fit_gpytorch_mll()`

In [49]:
train_X = torch.randn((10,1))
train_Y = torch.randn((10,3))

outcome_model = MultitaskGPModel(
    train_X,
    train_Y,
    latent_dim = 1,
    outcome_transform=Standardize(3),
).to(torch.double)
lcm_mll = ExactMarginalLogLikelihood(
    outcome_model.likelihood, outcome_model
)

fit_gpytorch_mll(lcm_mll)

/home/yz685/anaconda3/envs/bope_pca/lib/python3.9/site-packages/botorch/models/gpytorch.py:119: UserWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  warnings.warn(
/home/yz685/anaconda3/envs/bope_pca/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(


ExactMarginalLogLikelihood(
  (likelihood): MultitaskGaussianLikelihood(
    (raw_task_noises_constraint): GreaterThan(1.000E-04)
    (raw_noise_constraint): GreaterThan(1.000E-04)
  )
  (model): MultitaskGPModel(
    (likelihood): MultitaskGaussianLikelihood(
      (raw_task_noises_constraint): GreaterThan(1.000E-04)
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
    (mean_module): MultitaskMean(
      (base_means): ModuleList(
        (0): ConstantMean()
        (1): ConstantMean()
        (2): ConstantMean()
      )
    )
    (covar_module): LCMKernel(
      (covar_module_list): ModuleList(
        (0): MultitaskKernel(
          (task_covar_module): IndexKernel(
            (IndexKernelPrior): LKJCovariancePrior()
            (raw_var_constraint): Positive()
          )
          (data_covar_module): MaternKernel(
            (lengthscale_prior): GammaPrior()
            (raw_lengthscale_constraint): Positive()
          )
        )
      )
    )
  )
)

## data in `torch.double`, use `fit_gpytorch_mll_scipy()`

In [67]:
from botorch.optim.core import OptimizationStatus, OptimizationResult

In [3]:
train_X = torch.randn((10,3))
train_Y = torch.randn((10,10))

outcome_model = MultitaskGPModel(
    train_X.to(torch.double),
    train_Y.to(torch.double),
    latent_dim = 2,
    outcome_transform=Standardize(10),
).to(torch.double)

print('posterior mean before fitting', outcome_model.posterior(train_X).mean)

lcm_mll = ExactMarginalLogLikelihood(
    outcome_model.likelihood, outcome_model
)

fitting_success = False
num_tries = 0

while not fitting_success and num_tries <= 5:
    try:
        opt_result = fit_gpytorch_mll_scipy(lcm_mll, options={"maxls": 100})
        fitting_success = True
        break
    except:
        num_tries += 1
        continue

posterior mean before fitting tensor([[-0.2758, -0.7815,  0.8030,  0.7840,  0.0750, -0.7283, -0.5729,  0.4818,
         -0.2088, -0.3834],
        [ 0.4359,  1.0367, -0.0621,  0.8455, -0.2257,  0.8556, -0.4658, -0.1663,
         -0.0757,  0.1219],
        [-0.2982,  0.0540, -0.7276, -0.5150,  0.0779,  0.4909,  1.2590, -0.4509,
          0.6779,  1.4062],
        [ 0.1752,  0.5565, -0.4055, -0.0469,  0.4804,  0.9298,  0.2402,  0.0786,
         -0.5661,  0.1816],
        [-0.0674, -0.2898, -0.1221, -0.7062,  0.3479, -0.5137,  0.2359,  1.0690,
          0.9309,  0.6301],
        [-0.8671,  0.2030, -0.8960, -0.3402, -0.4537, -0.6126,  0.3092, -0.8132,
          0.2302, -0.2131],
        [-0.1571, -0.6594,  0.2735,  0.1825,  0.0404, -0.5214, -0.1548,  0.6475,
         -0.4969,  0.0916],
        [-0.4605, -0.5016,  0.8453,  0.2015, -0.3472, -0.1339, -0.1239, -0.2249,
         -0.1179, -0.4111],
        [ 1.3574,  0.3260,  0.6584, -0.6416,  0.5202,  0.6305, -0.0211,  0.4471,
         -0.0239,

/home/yz685/anaconda3/envs/bope_pca/lib/python3.9/site-packages/gpytorch/models/exact_gp.py:274: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(
/home/yz685/anaconda3/envs/bope_pca/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(
/home/yz685/anaconda3/envs/bope_pca/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  warnings.warn(
/home/yz685/anaconda3/envs/bope_pca/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/home/yz685/anaconda3/envs/bope_pca/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-05 to the diagonal
  warnings.warn(
/home/yz685/anaconda3/envs/bope_

In [5]:
opt_result

NameError: name 'opt_result' is not defined

## data in `torch.float`, use `fit_gpytorch_mll_scipy()`

In [84]:
train_X = torch.randn((10,1))
train_Y = torch.randn((10,3))

outcome_model = MultitaskGPModel(
    train_X,
    train_Y,
    latent_dim = 1,
    outcome_transform=Standardize(3),
).to(torch.double)
lcm_mll = ExactMarginalLogLikelihood(
    outcome_model.likelihood, outcome_model
)

fit_gpytorch_mll_scipy(lcm_mll)

/home/yz685/anaconda3/envs/bope_pca/lib/python3.9/site-packages/botorch/models/gpytorch.py:119: UserWarning: The model inputs are of type torch.float32. It is strongly recommended to use double precision in BoTorch, as this improves both precision and stability and can help avoid numerical errors. See https://github.com/pytorch/botorch/discussions/1444
  warnings.warn(
/home/yz685/anaconda3/envs/bope_pca/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(
/home/yz685/anaconda3/envs/bope_pca/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-07 to the diagonal
  warnings.warn(
/home/yz685/anaconda3/envs/bope_pca/lib/python3.9/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-06 to the diagonal
  warnings.warn(
/home/yz685/anaconda3/envs/bope_pca/lib/python3.9/site-packag

NotPSDError: Matrix not positive definite after repeatedly adding jitter up to 1.0e-03.